<a href="https://colab.research.google.com/github/TigistW/Deep-Learning-Labs/blob/main/DL_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [333]:
import torch

In [334]:
class DenseLayer:
    def __init__(self, n_features, n_neurons):
        self.weights = 0.01 * torch.rand(n_features, n_neurons)
        self.biases = torch.zeros(1, n_neurons)
        self.num_neurons = n_neurons
        self.output = None

    def forward(self, inputs):
        self.output = torch.matmul(inputs, self.weights) + self.biases


class Activation_ReLU:
    def forward(self, inputs):
        self.output = torch.max(torch.tensor(0.0), inputs)
class Activation_Linear:
    def forward(self, inputs):
        self.output = inputs

class Activation_Sigmoid:
    def forward(self, inputs):
        self.output = 1 / (1 + torch.exp(-inputs))

class Activation_Softmax:
    def forward(self, inputs):
        exp_values = torch.exp(inputs) - torch.max(inputs, axis=1, keepdims=True).values
        summed_exp = torch.sum(exp_values, axis=1, keepdims=True)
        self.output = exp_values / summed_exp

class Loss_CategoricalCrossentropy:
    def __init__(self, y_pred, y_true):
      self.loss = None
      # added clip to avoid undefined values
      y_pred = torch.clip(y_pred, 1e-8, 1 - 1e-8)

      if len(y_true.shape) == 2:
        self.forward_with_hot_encoding(y_pred, y_true)
      elif len(y_true.shape) == 1:
        self.forward_without_encoding(y_pred, y_true)
      else:
        pass

    def forward_without_encoding(self, y_pred, y_true):
        correct_confidences = y_pred[range(len(y_pred)), y_true]
        log_loss = -torch.log(correct_confidences)
        self.loss = torch.mean(log_loss)

    def forward_with_hot_encoding(self, y_pred, y_true):
        correct_confidences = torch.sum(y_pred * y_true, axis=1)
        log_loss = -torch.log(correct_confidences)
        self.loss = torch.mean(log_loss)

class NeuralNetwork:
    def __init__(self, input, num_features, num_neurons, activation_function):

        self.num_features = num_features
        self.activation = activation_function
        self.num_neurons = num_neurons
        self.input = input
        self.output = None
        self.layer = None

    def forward(self):
        self.layer = DenseLayer(self.num_features, self.num_neurons)
        self.layer.forward(self.input)
        self.activation.forward(self.layer.output)
        self.output = self.activation.output

class Accuracy:
  # included one hot encoded handling mech in this implementation
  def acc_percentage(self, predicted, target):
      total_samples = len(target)
      predictions = torch.argmax(predicted, axis=1)
      print("predictions", predictions[:10])
      if len(target.shape) == 2:
        target = torch.argmax(target, axis=1)
      print(target[:10])
      self.accuracy = torch.mean((predictions == target).float()) * 100.0
      print(self.accuracy)


In [335]:
def forward_pass(input):
  hidden_layer_1.forward(input)
  activation1.forward(hidden_layer_1.output)
  output_layer.forward(activation1.output)
  activation_2.forward(output_layer.output)
  return activation_2.output


In [336]:
def linear_derivative(x):
    return torch.ones_like(x)
def sigmoid_derivative(x):
    return x * (1 - x)

def back_prop(fp):

      lr = torch.tensor(0.01)

      # output layer gradients with linear activation
      output_delta_1 = (fp[0][0] - y[0]) * linear_derivative(fp[0][0])
      output_delta_2 = (fp[0][1] - y[1]) * linear_derivative(fp[0][1])

      # update output layer weights and biases
      for i in range(4):
          output_layer.weights[i][0] -= lr * output_delta_1 * activation1.output[0][i]
          output_layer.weights[i][1] -= lr * output_delta_2 * activation1.output[0][i]

      output_layer.biases[0][0] -= lr * output_delta_1
      output_layer.biases[0][1] -= lr * output_delta_2

      # hidden layer gradients
      hidden_delta_1 = sigmoid_derivative(hidden_layer_1.output[0][0]) * (
          output_delta_1 * output_layer.weights[0][0] + output_delta_2 * output_layer.weights[1][0]
      )
      hidden_delta_2 = sigmoid_derivative(hidden_layer_1.output[0][1]) * (
          output_delta_1 * output_layer.weights[0][1] + output_delta_2 * output_layer.weights[1][1]
      )
      hidden_delta_3 = sigmoid_derivative(hidden_layer_1.output[0][2]) * (
          output_delta_1 * output_layer.weights[2][0] + output_delta_2 * output_layer.weights[3][0]
      )
      hidden_delta_4 = sigmoid_derivative(hidden_layer_1.output[0][3]) * (
          output_delta_1 * output_layer.weights[2][1] + output_delta_2 * output_layer.weights[3][1]
      )

      # update hidden layer weights and biases
      for i in range(2):
          hidden_layer_1.weights[i][0] -= lr * hidden_delta_1 * X[i]
          hidden_layer_1.weights[i][1] -= lr * hidden_delta_2 * X[i]
          hidden_layer_1.weights[i][2] -= lr * hidden_delta_3 * X[i]
          hidden_layer_1.weights[i][3] -= lr * hidden_delta_4 * X[i]

      hidden_layer_1.biases[0][0] -= lr * hidden_delta_1
      hidden_layer_1.biases[0][1] -= lr * hidden_delta_2
      hidden_layer_1.biases[0][2] -= lr * hidden_delta_3
      hidden_layer_1.biases[0][3] -= lr * hidden_delta_4


In [337]:
def error_calculation(y_true, y_pred):
  return torch.mean(0.5*(y_true - y_pred)**2)

In [338]:
X = torch.tensor([0.1, 0.1])
y = torch.tensor([0.5, 0.8])

hidden_layer_1 = DenseLayer(2, 4)
activation1 = Activation_Sigmoid()
output_layer = DenseLayer(4, 2)
activation_2 = Activation_Linear()

In [339]:
loss = 0.00001

In [340]:
# calculate prediction and error of the prediction
y_pred = forward_pass(X)
err = error_calculation(y, y_pred)

print("Initial loss:", err, "Initial prediction", y_pred)

while err > loss:
  back_prop(y_pred)
  y_pred = forward_pass(X)
  err = error_calculation(y, y_pred)
print("Final loss:", err)
print("Final prediction:",y_pred)
print("Correct value:",y)

Initial loss: tensor(0.2164) Initial prediction tensor([[0.0100, 0.0092]])
Final loss: tensor(9.9913e-06)
Final prediction: tensor([[0.4967, 0.7946]])
Correct value: tensor([0.5000, 0.8000])
